In [1]:
## La version de python utilisée est 3.12.7

!pip install -r requirements.txt -q

In [2]:
import pandas as pd
import requests as rq
import lxml as lxml
from bs4 import BeautifulSoup
import io as io
import math
import gzip
import shutil
import os
import geopandas as gpd
import matplotlib.pyplot as plt
import folium
import json
from pandasgui import show
import numpy as np
from io import BytesIO
from folium.plugins import HeatMap
import nbconvert

from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut

In [3]:
# URL du fichier Excel
url_cotier = 'https://comersis.com/free-download.php?f=communes_littorales_2019.xlsx&d=littoral'

# Télécharger le fichier
response = rq.get(url_cotier)

# Lire le fichier Excel dans un DataFrame
df_cotieres = pd.read_excel(BytesIO(response.content))
df_cotieres = df_cotieres[df_cotieres['Région'] != "Départements d'Outre-Mer"]


print(df_cotieres.columns)
show(df_cotieres)

PandasGUI INFO — pandasgui.gui — Opening PandasGUI


Index(['Région', 'ID département', 'Nom département', 'Code Insee comm.',
       'Nom commune', 'Classement', 'Motif du classement',
       'nom de l'entité géographique dont le littoral est protégé',
       'Population', 'commune déleguée'],
      dtype='object')


C:\Users\flori\AppData\Local\Temp\ipykernel_14700\1155764729.py:13: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  show(df_cotieres)
C:\Users\flori\AppData\Local\Temp\ipykernel_14700\1155764729.py:13: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  show(df_cotieres)
C:\Users\flori\AppData\Local\Temp\ipykernel_14700\1155764729.py:13: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

C:\Users\flori\AppData\Local\Temp\ipykernel_14700\11557647

In [4]:
# Filtrer sur le Motif du classement
df_cotieres = df_cotieres[df_cotieres['Motif du classement'] == "Commune riveraine de la mer ou d'un océan"]

print(df_cotieres.columns)
liste_cotieres = df_cotieres['Nom commune'].unique().tolist()
print(liste_cotieres[1:10])
# Statistiques par "Région" pour les communes filtrées
stats_region_filtered = df_cotieres.groupby('Région').apply(
    lambda x: pd.Series({
        'nombre_communes': x['Nom commune'].nunique(),
        'population_totale': x['Population'].sum()
    })
).reset_index()

# Afficher les résultats
print(stats_region_filtered)

Index(['Région', 'ID département', 'Nom département', 'Code Insee comm.',
       'Nom commune', 'Classement', 'Motif du classement',
       'nom de l'entité géographique dont le littoral est protégé',
       'Population', 'commune déleguée'],
      dtype='object')
['Beaulieu-sur-Mer', 'Cagnes-sur-Mer', 'Cannes', "Cap-d'Ail", 'Eze', 'Mandelieu-la-Napoule', 'Menton', 'Nice', 'Roquebrune-Cap-Martin']
                       Région  nombre_communes  population_totale
0                    Bretagne              194             941329
1                       Corse               94             240732
2             Hauts de France               52             391192
3                   Normandie              202             710948
4          Nouvelle Aquitaine               81             510392
5                   Occitanie               27             237370
6            Pays-de-la-Loire               46             353413
7  Provence-Alpes-Côte d'Azur               53            2291162


C:\Users\flori\AppData\Local\Temp\ipykernel_14700\2019719742.py:8: DeprecationWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.



In [5]:
# Charger le fichier GeoJSON des communes de France depuis le lien GitHub
url_commune = 'https://raw.githubusercontent.com/gregoiredavid/france-geojson/master/communes.geojson'
response_commune = rq.get(url_commune)
communes_geojson = response_commune.json()

# Charger le GeoJSON dans un GeoDataFrame
gdf_communes = gpd.GeoDataFrame.from_features(communes_geojson['features'])
gdf_communes_cotieres = gdf_communes[gdf_communes['nom'].isin(liste_cotieres)]

# Calculer les centroïdes pour chaque commune
gdf_communes_cotieres['centroid'] = gdf_communes_cotieres.geometry.centroid
gdf_communes_cotieres['latitude'] = gdf_communes_cotieres['centroid'].y
gdf_communes_cotieres['longitude'] = gdf_communes_cotieres['centroid'].x
gdf_communes_cotieres = gdf_communes_cotieres.drop(columns=['centroid'])
df_centres = gdf_communes_cotieres[['nom', 'latitude', 'longitude']]

liste_commune = gdf_communes_cotieres['nom'].to_list()
show(gdf_communes_cotieres)

C:\Users\flori\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\geopandas\geodataframe.py:1819: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\flori\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\geopandas\geodataframe.py:1819: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\flori\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz

In [ ]:
# Exporte le fichier des communes cotières

gdf_communes_cotieres.to_csv('data/communes_cotieres.csv', index=False, encoding='utf-8-sig',sep=";")

In [3]:
# URL du fichier GeoJSON des départements de France
url_departement = "https://france-geojson.gregoiredavid.fr/repo/departements.geojson"

# Récupérer le fichier GeoJSON des départements de France
response = rq.get(url_departement)
departements_geojson = response.json()

# Créer une liste pour stocker les noms des départements
departement_names = []

# Extraire les noms des départements depuis les propriétés de chaque feature
for feature in departements_geojson['features']:
    departement_name = feature['properties']['nom']
    departement_names.append(departement_name)

# Créer un DataFrame avec les noms des départements
df_departements = pd.DataFrame(departement_names, columns=['departementName'])
    
# Afficher le DataFrame
print(df_departements)

          departementName
0                   Aisne
1                    Aube
2                Calvados
3                  Cantal
4            Eure-et-Loir
..                    ...
91           Loir-et-Cher
92                Mayenne
93     Meurthe-et-Moselle
94            Deux-Sèvres
95  Territoire de Belfort

[96 rows x 1 columns]
